# Building and Orgainizing More Complex Joins

### importing modules

In [1]:
import pandas as pd
import sqlite3

### Connecting to the Chinook Database

In [2]:
conn = sqlite3.connect('chinook.db')

In [3]:
cur = conn.cursor()

### Checking Out Table Names 

In [4]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


### Creating a Subquery Using a With Clause For Ease of Clarity

In [5]:
pd.read_sql_query("WITH playlist_info AS\
                        (SELECT pl.playlist_id AS playlist_id,\
                                pl.name AS playlist_name,\
                                t.name AS track_name,\
                                t.milliseconds/1000 AS track_length\
                         FROM playlist AS pl\
                         INNER JOIN playlist_track AS plt ON plt.playlist_id = pl.playlist_id\
                         INNER JOIN track as t ON t.track_id = plt.track_id)\
                    SELECT playlist_id,\
                           playlist_name,\
                           COUNT(track_name) AS number_of_tracks,\
                           SUM(track_length) AS length_in_seconds\
                    FROM playlist_info\
                    GROUP BY playlist_id", conn)

,playlist_id,playlist_name,number_of_tracks,length_in_seconds
0,1,Music,3290,876049
1,3,TV Shows,213,500987
2,5,90’s Music,1477,397970
3,8,Music,3290,876049
4,9,Music Videos,1,294
5,10,TV Shows,213,500987
6,11,Brazilian Music,39,9464
7,12,Classical,75,21736
8,13,Classical 101 - Deep Cuts,25,6742
9,14,Classical 101 - Next Steps,25,7565


### Creating a View "Customer_gt_90_dollars" for Multiple References

In [6]:
view_query_gt_90 = "CREATE VIEW customer_gt_90_dollars AS\
                        SELECT c.*\
                        FROM customer AS c\
                        INNER JOIN invoice as i ON i.customer_id = c.customer_id\
                        GROUP BY i.customer_id\
                        HAVING SUM(i.total) > 90"

### Executing the Query

In [15]:
cur.execute(view_query_gt_90)

### Checking to Make Sure the View Was Created

In [16]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'view'", conn)

,type,name,tbl_name,rootpage,sql
0,view,customer_gt_90_dollars,customer_gt_90_dollars,0,CREATE VIEW customer_gt_90_dollars AS ...


### Querying to See My Newly Created "Customer_gt_90_dollars" View of Customers Who Spent >90

In [17]:
pd.read_sql_query("SELECT * FROM customer_gt_90_dollars", conn).head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


### Creating a Query for a View Of Customers Who Live in the USA

In [18]:
query_view_usa = "CREATE VIEW customer_usa AS\
                  SELECT *\
                  FROM customer AS c\
                  WHERE c.country = 'USA'"

In [19]:
cur.execute(query_view_usa)

### Checking to See if Both VIEWs are Now Available

In [20]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'view'", conn)

,type,name,tbl_name,rootpage,sql
0,view,customer_gt_90_dollars,customer_gt_90_dollars,0,CREATE VIEW customer_gt_90_dollars AS ...
1,view,customer_usa,customer_usa,0,CREATE VIEW customer_usa AS S...


In [21]:
pd.read_sql_query("SELECT * FROM customer_usa", conn).head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
1,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
2,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
3,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
4,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


In [22]:
pd.read_sql_query("SELECT *\
                   FROM customer_gt_90_dollars\
                   UNION\
                   SELECT *\
                   FROM customer_usa", conn).head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4


### Writing a Query of How Many Customers Have Spent Over $90 and Live in the USA

In [23]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type = 'view'", conn)

,type,name,tbl_name,rootpage,sql
0,view,customer_gt_90_dollars,customer_gt_90_dollars,0,CREATE VIEW customer_gt_90_dollars AS ...
1,view,customer_usa,customer_usa,0,CREATE VIEW customer_usa AS S...


In [24]:
pd.read_sql_query("WITH cst_inter AS(\
                       SELECT * FROM customer_gt_90_dollars\
                       INTERSECT\
                       SELECT * FROM customer_usa)\
                   SELECT\
                       e.first_name || ' ' || e.last_name AS name,\
                       e.title AS title,\
                       COUNT(ci.customer_id) num_customers\
                   FROM employee AS e\
                   INNER JOIN cst_inter AS ci ON ci.support_rep_id = e.employee_id\
                   WHERE e.title = 'Sales Support Agent'\
                   GROUP BY e.employee_id\
                   ORDER BY name", conn)

,name,title,num_customers
0,Margaret Park,Sales Support Agent,2
1,Steve Johnson,Sales Support Agent,2


### Using 2 Subqueries and a WITH command to Find India Customers and Total Purchases

In [25]:
pd.read_sql_query('''
WITH
    customers_india AS
        (
        SELECT * FROM customer
        WHERE country = "India"
        ),
    sales_per_customer AS
        (
         SELECT
             customer_id,
             SUM(total) total
         FROM invoice
         GROUP BY 1
        )

SELECT
    ci.first_name || " " || ci.last_name customer_name,
    spc.total total_purchases
FROM customers_india ci
INNER JOIN sales_per_customer spc ON ci.customer_id = spc.customer_id
ORDER BY customer_name''', conn)

,customer_name,total_purchases
0,Manoj Pareek,111.87
1,Puja Srivastava,71.28


In [ ]:
### Querying the Max Purchases by Customer Per Country

In [26]:
pd.read_sql_query('''
WITH
    tot_pur AS
        (
        SELECT 
            c.country AS country,
            c.first_name || ' ' || c.last_name AS customer_name,
            i.customer_id customer_id, 
            SUM(i.total) AS total_purchases
        FROM invoice AS i
        INNER JOIN customer as c
        ON c.customer_id = i.customer_id
        GROUP BY i.customer_id
        )
SELECT tp.customer_name AS customer_name,
       tp.country as country,
       MAX(tp.total_purchases) as total_purchases
FROM tot_pur as tp
GROUP BY country
ORDER BY country ASC
''', conn)
    

,customer_name,country,total_purchases
0,Diego Gutiérrez,Argentina,39.60
1,Mark Taylor,Australia,81.18
2,Astrid Gruber,Austria,69.30
3,Daan Peeters,Belgium,60.39
4,Luís Gonçalves,Brazil,108.90
5,François Tremblay,Canada,99.99
6,Luis Rojas,Chile,97.02
7,František Wichterlová,Czech Republic,144.54
8,Kara Nielsen,Denmark,37.62
9,Terhi Hämäläinen,Finland,79.20
